In [ ]:
import pandas as pd
import geopandas as gpd
import rioxarray
import math
import shapely

from shapely.geometry import Point, MultiPoint

In [ ]:
sightlines = pd.read_pickle("my_results/method_1/d06_sightlines_geometries.pickle")
sightlines = sightlines[["start_point", "end_point", "sight_line_points"]]

# Convert tuples with coordinates to shapely geometry
Should eventually be deprecated by already using shapely geometries
in the first place.

In [ ]:
def convert_to_point(row):
    return Point(row)

In [ ]:
def convert_to_multipoint(row):
    if row:
        return MultiPoint(row)
    else:
        return None

In [ ]:
# Convert start and end point to geometry
tuple_columns = ["start_point", "end_point"]

for col in tuple_columns:
    sightlines[col+"_geom"] = sightlines[col].apply(convert_to_point)


# Convert sight line points to geometry
sightlines["sl_points"] = sightlines["sight_line_points"].apply(convert_to_multipoint)